In [17]:
import torch
from transformers import EfficientNetImageProcessor, EfficientNetForImageClassification
from PIL import Image
import math
from torch import nn
import distracted

from distracted.dataset_loader import dataset_loader

image = Image.open('data/imgs/train/c0/img_34.jpg')

preprocessor = EfficientNetImageProcessor.from_pretrained("google/efficientnet-b0")
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")

inputs = preprocessor(image, return_tensors="pt")

with torch.no_grad():
    logits = model(**inputs).logits

# model predicts one of the 1000 ImageNet classes
predicted_label = logits.argmax(-1).item()
print(model.config.id2label[predicted_label]),
# device = torch.device("cuda")
device = torch.device('cpu')
print(device)


seat belt, seatbelt
cpu


In [22]:
# model.config
for name, para in model.named_parameters():
    print(name)

efficientnet.embeddings.convolution.weight
efficientnet.embeddings.batchnorm.weight
efficientnet.embeddings.batchnorm.bias
efficientnet.encoder.blocks.0.depthwise_conv.depthwise_conv.weight
efficientnet.encoder.blocks.0.depthwise_conv.depthwise_norm.weight
efficientnet.encoder.blocks.0.depthwise_conv.depthwise_norm.bias
efficientnet.encoder.blocks.0.squeeze_excite.reduce.weight
efficientnet.encoder.blocks.0.squeeze_excite.reduce.bias
efficientnet.encoder.blocks.0.squeeze_excite.expand.weight
efficientnet.encoder.blocks.0.squeeze_excite.expand.bias
efficientnet.encoder.blocks.0.projection.project_conv.weight
efficientnet.encoder.blocks.0.projection.project_bn.weight
efficientnet.encoder.blocks.0.projection.project_bn.bias
efficientnet.encoder.blocks.1.expansion.expand_conv.weight
efficientnet.encoder.blocks.1.expansion.expand_bn.weight
efficientnet.encoder.blocks.1.expansion.expand_bn.bias
efficientnet.encoder.blocks.1.depthwise_conv.depthwise_conv.weight
efficientnet.encoder.blocks.1.d

In [7]:
# type(model(**inputs))

transformers.modeling_outputs.ImageClassifierOutputWithNoAttention

In [3]:
# dataset = dataset_loader()

[Link to guide for adding additional layers to pretrained model](https://medium.com/analytics-vidhya/how-to-add-additional-layers-in-a-pre-trained-model-using-pytorch-5627002c75a5)

towardsdatasciecne complete architectural details of all efficientnet models

EfficientNetExpansionLayer is Module 1
EfficientNetSqueezeExciteLayer is probaly module 3
EfficientNetFinalBlockLayer is probably module 5


[Paper on adapters](https://arxiv.org/pdf/1803.10082.pdf)


In [11]:
model = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")
from distracted.adapters import EfficientNetAdapterEncoding
from distracted.adapters import print_block_layout
from distracted.adapters import get_adapter_model
print_block_layout(model.config)
# adapter_encoder = EfficientNetAdapterEncoding(model.config)
# model.efficientnet.encoder = adapter_encoder
adapter_locations = [(2,4),(4,10)]
model = get_adapter_model(base_model_suffix="b0", adapter_locations=adapter_locations)



Base block 0: [(32, 16)] valid block numbers: (0, 0)
Base block 1: [(16, 24), (24, 24)] valid block numbers: (1, 2)
Base block 2: [(24, 40), (40, 40)] valid block numbers: (2, 4)
Base block 3: [(40, 80), (80, 80), (80, 80)] valid block numbers: (3, 7)
Base block 4: [(80, 112), (112, 112), (112, 112)] valid block numbers: (4, 10)
Base block 5: [(112, 192), (192, 192), (192, 192), (192, 192)] valid block numbers: (5, 14)
Base block 6: [(192, 320)] valid block numbers: (6, 15)
Avoid block numbers [] since they have adjust_padding=True


In [16]:
# for para in model.named_parameters():
#     print(para[0],para[1].requires_grad)


num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(num_params)
# for para in model.parameters():
#     print(para.numel())

5305198


In [9]:
print(len(encoder_instance.blocks))
encoder_instance.blocks[0].named_parameters

16


<bound method Module.named_parameters of EfficientNetBlock(
  (depthwise_conv): EfficientNetDepthwiseLayer(
    (depthwise_conv_pad): ZeroPad2d((0, 1, 0, 1))
    (depthwise_conv): EfficientNetDepthwiseConv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=same, groups=32, bias=False)
    (depthwise_norm): BatchNorm2d(32, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (depthwise_act): SiLUActivation()
  )
  (squeeze_excite): EfficientNetSqueezeExciteLayer(
    (squeeze): AdaptiveAvgPool2d(output_size=1)
    (reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1), padding=same)
    (act_reduce): SiLUActivation()
    (act_expand): Sigmoid()
  )
  (projection): EfficientNetFinalBlockLayer(
    (project_conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), padding=same, bias=False)
    (project_bn): BatchNorm2d(16, eps=0.001, momentum=0.99, affine=True, track_running_stats=True)
    (d

In [121]:
# encoder_instance.blocks[0]

In [55]:
# adapter_encoding = EfficientNetAdapterEncoding(model)
print(len(encoder_instance.blocks))
block_input = torch.randn(1,40,300,300)
encoder_block_output = encoder_instance.blocks[4](block_input)
print(encoder_block_output.size())
# print(adapter_encoding.adapters[0])
# adapter_encoding.adapters[0](encoder_block_output)
# for block in encoder_instance.blocks:
#     print(type(block))

encoder_block_output.view(encoder_block_output.size(0),-1).shape

16
torch.Size([1, 40, 300, 300])


torch.Size([1, 3600000])

In [10]:
num_params = sum(p.numel() for p in encoder_instance.blocks[4].parameters() if p.requires_grad)
print(f"Number of trainable parameters: {num_params}")

Number of trainable parameters: 31290


In [7]:
len(encoder_instance.blocks[0].)

AttributeError: 'EfficientNetBlock' object has no attribute 'size'

In [9]:
encoder_instance = model.efficientnet.encoder
encoder_instance.to(device)
# adapter_encoding.to("cpu")
from torchsummary import summary
# use this later for checking trainable params etc
# block 0 has 64, 300, 300
block = encoder_instance.blocks[0]
block.to(device)
summary(block,input_size=(64,300,300))

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument weight in method wrapper_CUDA___conv_depthwise2d)

In [92]:
model.efficientnet.embeddings(**inputs).shape

torch.Size([1, 64, 300, 300])

In [15]:
# Loop through named_parameters and set adapters to True and everything else to False?
for name,para in model.named_parameters():
    print(name, para.requires_grad)

efficientnet.embeddings.convolution.weight True
efficientnet.embeddings.batchnorm.weight True
efficientnet.embeddings.batchnorm.bias True
efficientnet.encoder.blocks.0.depthwise_conv.depthwise_conv.weight True
efficientnet.encoder.blocks.0.depthwise_conv.depthwise_norm.weight True
efficientnet.encoder.blocks.0.depthwise_conv.depthwise_norm.bias True
efficientnet.encoder.blocks.0.squeeze_excite.reduce.weight True
efficientnet.encoder.blocks.0.squeeze_excite.reduce.bias True
efficientnet.encoder.blocks.0.squeeze_excite.expand.weight True
efficientnet.encoder.blocks.0.squeeze_excite.expand.bias True
efficientnet.encoder.blocks.0.projection.project_conv.weight True
efficientnet.encoder.blocks.0.projection.project_bn.weight True
efficientnet.encoder.blocks.0.projection.project_bn.bias True
efficientnet.encoder.blocks.1.expansion.expand_conv.weight True
efficientnet.encoder.blocks.1.expansion.expand_bn.weight True
efficientnet.encoder.blocks.1.expansion.expand_bn.bias True
efficientnet.encod

In [13]:
model.config

EfficientNetConfig {
  "_name_or_path": "google/efficientnet-b0",
  "architectures": [
    "EfficientNetForImageClassification"
  ],
  "batch_norm_eps": 0.001,
  "batch_norm_momentum": 0.99,
  "depth_coefficient": 1.0,
  "depth_divisor": 8,
  "depthwise_padding": [],
  "drop_connect_rate": 0.2,
  "dropout_rate": 0.2,
  "expand_ratios": [
    1,
    6,
    6,
    6,
    6,
    6,
    6
  ],
  "hidden_act": "swish",
  "hidden_dim": 1280,
  "id2label": {
    "0": "tench, Tinca tinca",
    "1": "goldfish, Carassius auratus",
    "2": "great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias",
    "3": "tiger shark, Galeocerdo cuvieri",
    "4": "hammerhead, hammerhead shark",
    "5": "electric ray, crampfish, numbfish, torpedo",
    "6": "stingray",
    "7": "cock",
    "8": "hen",
    "9": "ostrich, Struthio camelus",
    "10": "brambling, Fringilla montifringilla",
    "11": "goldfinch, Carduelis carduelis",
    "12": "house finch, linnet, Carpodacus mexicanus

In [6]:
# adapter_encoder.adapters[4].named_parameters
# for para in adapter_encoder.adapters[4].named_parameters():
#     print(para[0],para[1].requires_grad)


In [5]:
# model7 = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b7")
# model0 = EfficientNetForImageClassification.from_pretrained("google/efficientnet-b0")
# print(model0.config, model7.config)